In [1]:
import pandas as pd
import numpy as np

In [17]:
from tkinter import *
from tkinter import filedialog
import tkinter.messagebox as mbox
import os
import shutil



excel_ext=r"*.xlsx *.xks *.csv"

def file_find():
    file = filedialog.askopenfilenames(filetypes=(("Excel file", excel_ext), ("all file", "*.*")), initialdir="C:\\Users\\Administrator\\Desktops")
    en_filepath.delete(0,END)
    en_filepath.insert(END, file[0])

def file_upload():
    if len(en_filepath.get())==0:
        mbox.showinfo("Warning", "select file, please")
        return
    else:
        file_name = os.path.basename(en_filepath.get())
        dest_path = os.path.join("C:\\Users\\Administrator\\Desktop", file_name)
        shutil.copy(en_filepath.get(), dest_path)
        en_filepath.delete(0, END)
        return


root = Tk()

root.title("출결 데이터 정제 프로그램")
root.geometry("500x200+200+200")
root.resizable(True, True)

en_filepath = Entry(root, width=100)
en_filepath.pack(fill="x", padx=1, pady=1)

label = Label(root, text='Hello World')
label.pack()

btn_find = Button(root, text="find", width=10, command=file_find)
btn_find.pack(side="right", padx=1, pady=1)
btn_upload = Button(root, text="upload", width=10, command=file_upload)
btn_upload.pack(side="right", padx=1, pady=1)




root.mainloop()

Exception in Tkinter callback
Traceback (most recent call last):
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\tkinter\__init__.py", line 1948, in __call__
    return self.func(*args)
           ^^^^^^^^^^^^^^^^
  File "C:\Users\Administrator\AppData\Local\Temp\ipykernel_5084\3330365540.py", line 23, in file_upload
    shutil.copy(en_filepath.get(), dest_path)
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\shutil.py", line 419, in copy
    copyfile(src, dst, follow_symlinks=follow_symlinks)
  File "c:\Users\Administrator\AppData\Local\Programs\Python\Python311\Lib\shutil.py", line 258, in copyfile
    with open(dst, 'wb') as fdst:
         ^^^^^^^^^^^^^^^
FileNotFoundError: [Errno 2] No such file or directory: 'C:\\Users\\Administrator\\Desktop\\result\\학습이력 정보 ai.xlsx'


In [2]:
ai_df = pd.read_excel(r'C:\Users\Administrator\Desktop\error\code\학습이력 정보 AI.xlsx')
ai_drop_df = ai_df.drop(['과목명','훈련기관 PK', '훈련기관과정 PK','훈련기관수업PK','이수 누계(초)','접근IP'], axis=1)
print(ai_drop_df.head(3))
ai_drop_df = ai_drop_df.drop(0, axis=0)

iot_df = pd.read_excel(r'C:\Users\Administrator\Desktop\error\code\학습이력 정보 iot.xlsx')
iot_drop_df = iot_df.drop(['과목명','훈련기관 PK', '훈련기관과정 PK','훈련기관수업PK','이수 누계(초)','접근IP'], axis=1)
print(iot_drop_df.head(3))
iot_drop_df = iot_drop_df.drop(0, axis=0)

semi_df = pd.read_excel(r'C:\Users\Administrator\Desktop\error\code\학습이력 정보 반도체.xlsx')
semi_drop_df = semi_df.drop(['과목명','훈련기관 PK', '훈련기관과정 PK','훈련기관수업PK','이수 누계(초)','접근IP'], axis=1)
print(semi_drop_df.head(3))
semi_drop_df = semi_drop_df.drop(0, axis=0)

  훈련생 성명      훈련기관회원 PK       차시구분          학습기준일        start   start time  \
0    NaN  USER_AGENT_PK  EVAL_TYPE  SUBMIT_DUE_DT          NaN          NaN   
1    전용석     2016212039    파이썬_1차시       20220926  2022.11.20    오후 1:51:44   
2    하준한     2020212041    파이썬_1차시       20220926  2022.11.20    오후 4:35:34   

                  학습시작시간                 학습종료시간  강의 이수기간(분)    강의 시간(초) 진도율(%)  
0      SUBMIT_DATE_START            SUBMIT_DATE         NaN  STUDY_TIME  SCORE  
1  2022.11.20 오후 1:51:44  2022.11.20 오후 2:17:32        25.0        3600    100  
2  2022.11.20 오후 4:35:34  2022.11.20 오후 4:43:12         7.0        1099  30.53  
  훈련생 성명      훈련기관회원 PK       차시구분          학습기준일        start   start time  \
0    NaN  USER_AGENT_PK  EVAL_TYPE  SUBMIT_DUE_DT          NaN          NaN   
1    정욱현   wjddnrgus908    파이썬_6차시       20220926  2022.10.26    오후 6:25:00   
2    신경호     2018148018  클라이언트_1차시       20220930  2022.11.08   오후 11:42:03   

                   학습시작시간                 

In [3]:
#0.0분 이수 데이터 삭제
ai_df=ai_drop_df[~ai_drop_df['강의 이수기간(분)'].isin([0.0])]
iot_df=iot_drop_df[~iot_drop_df['강의 이수기간(분)'].isin([0.0])]
semi_drop_df=semi_drop_df[~semi_drop_df['강의 이수기간(분)'].isin([0.0])]

# 2단위기간 이전 데이터 삭제
target_ai = ai_drop_df['학습기준일'].ge(20221026)
ai_drop_df=ai_drop_df[target_ai]

target_iot = iot_drop_df['학습기준일'].ge(20221026)
iot_drop_df=iot_drop_df[target_iot]

target = semi_drop_df['학습기준일'].ge(20221026)
semi_drop_df=semi_drop_df[target]

In [4]:
# 단위기간 최종 출결 
ai_check_df = ai_drop_df.groupby(['차시구분','훈련생 성명']).max()

ai_check_df = ai_check_df.groupby(['학습기준일', '훈련생 성명']).sum()
ai_check_df = ai_check_df.sort_values(by=['훈련생 성명', '학습기준일'])

ai_check_df['이수시간(초)']=(ai_check_df['진도율(%)']*36).round(0)
ai_check_df['이수시간(초)'].astype(int)

ai_check_df.to_excel(r'C:\Users\Administrator\Desktop\error\code\(result)온라인출석 AI.xlsx')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16900\3545598632.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ai_check_df = ai_check_df.groupby(['학습기준일', '훈련생 성명']).sum()


In [5]:
# 단위기간 최종 출결 
iot_check_df = iot_drop_df.groupby(['차시구분','훈련생 성명']).max()

iot_check_df = iot_check_df.groupby(['학습기준일', '훈련생 성명']).sum()
iot_check_df = iot_check_df.sort_values(by=['훈련생 성명', '학습기준일'])

iot_check_df['이수시간(초)']=(iot_check_df['진도율(%)']*36).round(0)
iot_check_df['이수시간(초)'].astype(int)

iot_check_df.to_excel(r'C:\Users\Administrator\Desktop\error\code\(result)온라인출석 iot.xlsx')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16900\1160088811.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  iot_check_df = iot_check_df.groupby(['학습기준일', '훈련생 성명']).sum()


In [6]:
# 단위기간 최종 출결 
semi_check_df = semi_drop_df.groupby(['차시구분','훈련생 성명']).max()

semi_check_df = semi_check_df.groupby(['학습기준일', '훈련생 성명']).sum()
semi_check_df = semi_check_df.sort_values(by=['훈련생 성명', '학습기준일'])

semi_check_df['이수시간(초)']=(semi_check_df['진도율(%)']*36).round(0)
semi_check_df['이수시간(초)'].astype(int)

semi_check_df.to_excel(r'C:\Users\Administrator\Desktop\error\code\(result)온라인출석 반도체.xlsx')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16900\2800209538.py:4: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  semi_check_df = semi_check_df.groupby(['학습기준일', '훈련생 성명']).sum()


In [7]:
# 출결이벤트 대상자 찾기

# 학습시작시간에서 점 삭제
ai_drop_df.replace(to_replace=r'[.]',value="", regex=True, inplace=True)

# 학습 시작일, 시작시간 구분
target_date_ai=ai_drop_df['학습시작시간'].str.slice(0, 8) 
ai_drop_df.insert(4,'시작일', target_date_ai)
ai_perfact_df=ai_drop_df

ai_perfact_df = ai_perfact_df.groupby(['훈련생 성명', '차시구분']).max()
ai_perfact_df = ai_perfact_df.groupby(['훈련생 성명','학습기준일', '시작일']).sum()

ai_perfact_df.to_excel(r'C:\Users\Administrator\Desktop\error\code\(result)퍼펙트출석 ai.xlsx')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16900\2855906606.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ai_perfact_df = ai_perfact_df.groupby(['훈련생 성명','학습기준일', '시작일']).sum()


In [8]:
# 출결이벤트 대상자 찾기

# 학습시작시간에서 점 삭제
iot_drop_df.replace(to_replace=r'[.]',value="", regex=True, inplace=True)

# 학습 시작일, 시작시간 구분
target_date_iot=iot_drop_df['학습시작시간'].str.slice(0, 8) 
iot_drop_df.insert(4,'시작일', target_date_iot)
iot_perfact_df=iot_drop_df

iot_perfact_df = iot_perfact_df.groupby(['훈련생 성명', '차시구분']).max()
iot_perfact_df = iot_perfact_df.groupby(['훈련생 성명','학습기준일', '시작일']).sum()

iot_perfact_df.to_excel(r'C:\Users\Administrator\Desktop\error\code\(result)퍼펙트출석 iot.xlsx')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16900\3601121957.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  iot_perfact_df = iot_perfact_df.groupby(['훈련생 성명','학습기준일', '시작일']).sum()


In [9]:
# 출결이벤트 대상자 찾기

# 학습시작시간에서 점 삭제
semi_drop_df.replace(to_replace=r'[.]',value="", regex=True, inplace=True)

# 학습 시작일, 시작시간 구분
target_date_semi=semi_drop_df['학습시작시간'].str.slice(0, 8) 
semi_drop_df.insert(4,'시작일', target_date_semi)
semi_perfact_df=semi_drop_df

semi_perfact_df = semi_perfact_df.groupby(['훈련생 성명', '차시구분']).max()
semi_perfact_df = semi_perfact_df.groupby(['훈련생 성명','학습기준일', '시작일']).sum()

semi_perfact_df.to_excel(r'C:\Users\Administrator\Desktop\error\code\(result)퍼펙트출석 반도체.xlsx')

C:\Users\Administrator\AppData\Local\Temp\ipykernel_16900\3713269251.py:12: FutureWarning: The default value of numeric_only in DataFrameGroupBy.sum is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  semi_perfact_df = semi_perfact_df.groupby(['훈련생 성명','학습기준일', '시작일']).sum()
